In [1]:
# HW1Q3.ipynb
# Date: 07/30/2021
# Author: Rio Weil
# Functionality: Calculates the concatenation threshold for the 9-qubit Shor Code under independent depolarizing Pauli noise analytically.
# Note: The errors under concatenation are found by composing the errors from the 3-qubit bit flip and phase flip codes (efficient).

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import *

Although a complete solution has already been found, we wish to find a more efficient solution; one that could be performed on a programming calculator (say, with 64kB of memory only). Here, we offer a much more efficient solution. We start by (as before) defining some functions to check the error type of a given length 3 Pauli operator (under the bit flip code):

In [3]:
def checkx(q1, q2, q3):
    if q1 == "X" or q1 == "Y":
        x1 = 1
    else:
        x1 = 0
    if q2 == "X" or q2 == "Y":
        x2 = 1
    else:
        x2 = 0
    if q3 == "X" or q3 == "Y":
        x3 = 1
    else:
        x3 = 0
    if (x1 + x2 + x3) > 1:
        return True
    else:
        return False

def checkz(q1, q2, q3):
    if q1 == "Z" or q1 == "Y":
        z1 = 1
    else:
        z1 = 0
    if q2 == "Z" or q2 == "Y":
        z2 = 1
    else:
        z2 = 0
    if q3 == "Z" or q3 == "Y":
        z3 = 1
    else:
        z3 = 0
    return ((z1 + z2 + z3) % 2 == 1)

Now for where we have our exponential time save; instead of checking and categorizing $4^9$ cases, we only check 64; we iterate through all possible length $3$ Pauli operators, and check if under the bit flip code we get an $X$, $Y$, or $Z$ error (or no error at all).

In [4]:
errors = ["I", "X", "Y", "Z"]
pxlist = []
pylist = []
pzlist = []

for a in errors:
    for b in errors:
        for c in errors:
            error = a + b + c
            xerr = checkx(a, b, c)
            zerr = checkz(a, b, c)
            if xerr:
                if zerr:
                    pylist.append(error)
                else:
                    pxlist.append(error)
            else:
                if zerr:
                    pzlist.append(error)         

As before, we use a dictionary to group similar error types together.

In [5]:
def create_anagrams_list(wordlist):
    sortedlist = []
    for i in range(len(wordlist)):
        wordlist[i] = sorted(wordlist[i])
    while (len(wordlist) != 0):
        word = wordlist[0]
        count = 0
        for i in range(len(wordlist)):
            if (word == wordlist[i]):
                count += 1
        for i in range(count):
            wordlist.remove(word)
        sortedlist.append([word, count])   
    return sortedlist

In [6]:
def process_list(almost_done):
    for i in range(len(almost_done)):
        word = ""
        for j in range(3):
            word += almost_done[i][0][j]
        almost_done[i] = [word, almost_done[i][1]]

In [7]:
sorted_pxlist = create_anagrams_list(pxlist)
sorted_pylist = create_anagrams_list(pylist)
sorted_pzlist = create_anagrams_list(pzlist)

In [8]:
process_list(sorted_pxlist)
process_list(sorted_pylist)
process_list(sorted_pzlist)

We now have the error type and multiplicities for each of the error types.

In [9]:
print(sorted_pxlist)
print(sorted_pylist)
print(sorted_pzlist)

[['IXX', 3], ['IYY', 3], ['XXX', 1], ['XYY', 3], ['XYZ', 6]]
[['IXY', 6], ['XXY', 3], ['XXZ', 3], ['YYY', 1], ['YYZ', 3]]
[['IIY', 3], ['IIZ', 3], ['IXZ', 6], ['YZZ', 3], ['ZZZ', 1]]


Now, we move to the symbolic computing section. Here we let $p_x = x$, $p_y = y$, $p_z = z$.

In [10]:
x, y, z = symbols("x y z")

The below function converts the above error type/multiplicity list into an expression for $p_x', p_y', p_z'$ under the bit flip code (i.e. what does each type of error look like after one round of concatenation?)

In [11]:
def make_symbolic(sortedlist):
    finalexpr = 0
    for i in range(len(sortedlist)):
        string = sortedlist[i][0]
        expr = sortedlist[i][1]
        for j in range(len(string)):
            if string[j] == "X":
                expr *= x
            elif string[j] == "Y":
                expr *= y
            elif string[j] == "Z":
                expr *= z
            else:
                expr *= (1 - x - y - z)
        finalexpr += expr
    return finalexpr

In [14]:
px_bf = make_symbolic(sorted_pxlist)
py_bf = make_symbolic(sorted_pylist)
pz_bf = make_symbolic(sorted_pzlist)

After this processing, we obtain $p_x', p_y', p_z'$ for the bit flip code to be:

In [15]:
simplify(px_bf)

x**3 + 3*x**2*(-x - y - z + 1) + 3*x*y**2 + 6*x*y*z + 3*y**2*(-x - y - z + 1)

In [16]:
simplify(py_bf)

3*x**2*y + 3*x**2*z - 6*x*y*(x + y + z - 1) + y**3 + 3*y**2*z

In [17]:
simplify(pz_bf)

-6*x*z*(x + y + z - 1) + 3*y*z**2 + 3*y*(x + y + z - 1)**2 + z**3 + 3*z*(x + y + z - 1)**2

Now, the result of applying the bit flip and phase flip codes <b>will be the same up to choice of symbols, by the symmetry of the codes</b>. This yields another large computational timesave. To obtain $p_{ypf}$, simply switch $x$ and $z$ in $p_{ybf}$. For $p_{xpf}$, switch $x$ and $z$ in $p_{zbf}$. Finally, for $p_{zbf}$, switch $x$ and $z$ in $p_{xbf}$.

In [18]:
t = symbols("t") # Temporary variable for substitution

In [19]:
px_pf = pz_bf.subs([(x, t), (z, x)]).subs(t, z)
py_pf = py_bf.subs([(x, t), (z, x)]).subs(t, z)
pz_pf = px_bf.subs([(x, t), (z, x)]).subs(t, z)

After this insightful substitution, we obtain $p_x', p_y', p_z'$ for the phase flip code to be:

In [20]:
simplify(px_pf)

x**3 + 3*x**2*y - 6*x*z*(x + y + z - 1) + 3*x*(x + y + z - 1)**2 + 3*y*(x + y + z - 1)**2

In [21]:
simplify(py_pf)

3*x*y**2 + 3*x*z**2 + y**3 + 3*y*z**2 - 6*y*z*(x + y + z - 1)

In [22]:
simplify(pz_pf)

6*x*y*z + 3*y**2*z + 3*y**2*(-x - y - z + 1) + z**3 + 3*z**2*(-x - y - z + 1)

Now, we want to determine how the error evolves under the Shor code. This is where our second step of theoretical insight comes in. <b> We substitute the expression of $p_{xbf}'$ into $p_{xpf}'$, $p_{ybf}'$ into $p_{ypf}'$, and $p_{zbf}'$ into $p_{zpf}'$. This is because the Shor code is exactly the concatenation of the bit flip code and the phase flip code; hence we can concatenate the errors in this way as well. </b> Making this subtitution, we have:

In [23]:
a, b, c = symbols("a, b, c") # Temporary variables

In [24]:
px_pf = px_pf.subs([(x, a), (y, b), (z, c)])
py_pf = py_pf.subs([(x, a), (y, b), (z, c)])
pz_pf = pz_pf.subs([(x, a), (y, b), (z, c)])

In [25]:
px_tot = px_pf.subs([(a, px_bf), (b, py_bf), (c, pz_bf)])
py_tot = py_pf.subs([(a, px_bf), (b, py_bf), (c, pz_bf)])
pz_tot = pz_pf.subs([(a, px_bf), (b, py_bf), (c, pz_bf)])

In [28]:
simplify(px_tot)

-6*(x**3 + 3*x**2*(-x - y - z + 1) + 3*x*y**2 + 6*x*y*z + 3*y**2*(-x - y - z + 1))*(-6*x*z*(x + y + z - 1) + 3*y*z**2 + 3*y*(x + y + z - 1)**2 + z**3 + 3*z*(x + y + z - 1)**2)*(x**3 + 3*x**2*y + 3*x**2*z + 3*x**2*(-x - y - z + 1) + 3*x*y**2 + 6*x*y*z - 6*x*y*(x + y + z - 1) - 6*x*z*(x + y + z - 1) + y**3 + 3*y**2*z + 3*y**2*(-x - y - z + 1) + 3*y*z**2 + 3*y*(x + y + z - 1)**2 + z**3 + 3*z*(x + y + z - 1)**2 - 1) + 3*(x**3 + 3*x**2*(-x - y - z + 1) + 3*x*y**2 + 6*x*y*z + 3*y**2*(-x - y - z + 1))*(x**3 + 3*x**2*y + 3*x**2*z + 3*x**2*(-x - y - z + 1) + 3*x*y**2 + 6*x*y*z - 6*x*y*(x + y + z - 1) - 6*x*z*(x + y + z - 1) + y**3 + 3*y**2*z + 3*y**2*(-x - y - z + 1) + 3*y*z**2 + 3*y*(x + y + z - 1)**2 + z**3 + 3*z*(x + y + z - 1)**2 - 1)**2 + (x**3 - 3*x**2*(x + y + z - 1) + 3*x*y**2 + 6*x*y*z - 3*y**2*(x + y + z - 1))**3 + 3*(x**3 - 3*x**2*(x + y + z - 1) + 3*x*y**2 + 6*x*y*z - 3*y**2*(x + y + z - 1))**2*(3*x**2*y + 3*x**2*z - 6*x*y*(x + y + z - 1) + y**3 + 3*y**2*z) + 3*(3*x**2*y + 3*x**2*z 

In [26]:
px_py = px_tot + py_tot
py_pz = py_tot + pz_tot

In [27]:
simplify(px_py)

-32*x**9 - 288*x**8*y + 144*x**8 - 1152*x**7*y**2 + 1152*x**7*y - 216*x**7 - 2688*x**6*y**3 + 4032*x**6*y**2 - 1512*x**6*y + 84*x**6 - 4032*x**5*y**4 + 8064*x**5*y**3 - 4536*x**5*y**2 + 504*x**5*y + 72*x**5 - 4032*x**4*y**5 + 10080*x**4*y**4 - 7560*x**4*y**3 + 1260*x**4*y**2 + 360*x**4*y - 54*x**4 - 2688*x**3*y**6 + 8064*x**3*y**5 - 7560*x**3*y**4 + 1680*x**3*y**3 + 720*x**3*y**2 - 216*x**3*y - 6*x**3 - 1152*x**2*y**7 + 4032*x**2*y**6 - 4536*x**2*y**5 + 1260*x**2*y**4 + 720*x**2*y**3 - 324*x**2*y**2 - 18*x**2*y + 9*x**2 - 288*x*y**8 + 1152*x*y**7 - 1512*x*y**6 + 504*x*y**5 + 360*x*y**4 - 216*x*y**3 - 18*x*y**2 + 18*x*y - 32*y**9 + 144*y**8 - 216*y**7 + 84*y**6 + 72*y**5 - 54*y**4 - 6*y**3 + 9*y**2

In [76]:
simplify(py_pz)

-128*y**9 - 1152*y**8*z + 576*y**8 - 4608*y**7*z**2 + 4608*y**7*z - 1152*y**7 - 10752*y**6*z**3 + 16128*y**6*z**2 - 8064*y**6*z + 1344*y**6 - 16128*y**5*z**4 + 32256*y**5*z**3 - 24192*y**5*z**2 + 8064*y**5*z - 1008*y**5 - 16128*y**4*z**5 + 40320*y**4*z**4 - 40320*y**4*z**3 + 20160*y**4*z**2 - 5040*y**4*z + 504*y**4 - 10752*y**3*z**6 + 32256*y**3*z**5 - 40320*y**3*z**4 + 26880*y**3*z**3 - 10080*y**3*z**2 + 2016*y**3*z - 162*y**3 - 4608*y**2*z**7 + 16128*y**2*z**6 - 24192*y**2*z**5 + 20160*y**2*z**4 - 10080*y**2*z**3 + 3024*y**2*z**2 - 486*y**2*z + 27*y**2 - 1152*y*z**8 + 4608*y*z**7 - 8064*y*z**6 + 8064*y*z**5 - 5040*y*z**4 + 2016*y*z**3 - 486*y*z**2 + 54*y*z - 128*z**9 + 576*z**8 - 1152*z**7 + 1344*z**6 - 1008*z**5 + 504*z**4 - 162*z**3 + 27*z**2

In [29]:
factor(px_py)

-(x + y)**2*(2*x + 2*y - 3)*(16*x**6 + 96*x**5*y - 48*x**5 + 240*x**4*y**2 - 240*x**4*y + 36*x**4 + 320*x**3*y**3 - 480*x**3*y**2 + 144*x**3*y + 12*x**3 + 240*x**2*y**4 - 480*x**2*y**3 + 216*x**2*y**2 + 36*x**2*y - 18*x**2 + 96*x*y**5 - 240*x*y**4 + 144*x*y**3 + 36*x*y**2 - 36*x*y + 16*y**6 - 48*y**5 + 36*y**4 + 12*y**3 - 18*y**2 + 3)

In [78]:
factor(py_pz)

-(y + z)**2*(4*y**2 + 8*y*z - 6*y + 4*z**2 - 6*z + 3)**2*(8*y**3 + 24*y**2*z - 12*y**2 + 24*y*z**2 - 24*y*z + 6*y + 8*z**3 - 12*z**2 + 6*z - 3)

Making the substitution of $x + y \mapsto p$, $y + z \mapsto q$, we get:

In [80]:
p, q = symbols("p, q")

In [84]:
newp = -p**2 * (2*p - 3) * ( 16 * p**6 -48 * p**5 + 36 * p**4 + 12 * p**3 - 18 * p**2 + 3)
expand(newp)

-32*p**9 + 144*p**8 - 216*p**7 + 84*p**6 + 72*p**5 - 54*p**4 - 6*p**3 + 9*p**2

In [85]:
newq = -q**2 * (4*q**2 - 6 * q + 3)**2 * (8 * q**3 - 12*q**2 + 6*q - 3)
expand(newq)

-128*q**9 + 576*q**8 - 1152*q**7 + 1344*q**6 - 1008*q**5 + 504*q**4 - 162*q**3 + 27*q**2

And these are identical to the expressions we obtained with the previous (inefficient) method. This method introduced here has a much more efficient runtime (we only check 64 cases; definitely feasible with a pocket calculator!) but yields the same result. If anything, this whole substitution business could be worked out by hand in desperate times, so if we were trapped in a cave with nothing but a programming calculator and pens/paper, this would definitely be solvable.